In [1]:
import pandas as pd

In [2]:
withdraw_transactions_df = pd.read_csv("../data/complete_withdraw_txs.csv")

In [3]:
lighter_withdraw_transactions_df = withdraw_transactions_df.drop(columns=["input"])

In [4]:
lighter_withdraw_transactions_df.to_csv("../data/lighter_complete_withdraw_txs.csv")

In [5]:
lighter_withdraw_transactions_df

,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
0,7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,10382378,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
1,8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,7303106,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
2,9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,3211601,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
3,13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,11795370,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
4,15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,9771737,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83777,181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,20743650,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
83778,181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,16358129,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
83779,181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,982698,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
83780,181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570459,81048817558,18511920,...,1,2021-09-04 22:53:33 UTC,13162025,0x15a807c50ecea90cd58f5f9d6a26e0c60f3e9f06dfa6...,1.349098e+11,3.899014e+09,2.0,81048817558,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47


In [6]:
withdraw_transactions_df

,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,...,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
0,7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb...,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,0x21a0adb6000000000000000000000000000000000000...,...,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be9...,NaN,NaN,NaN,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
1,8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da...,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,0x21a0adb6000000000000000000000000000000000000...,...,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac06...,NaN,NaN,NaN,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
2,9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f...,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,0x21a0adb6000000000000000000000000000000000000...,...,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290...,NaN,NaN,NaN,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
3,13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad1...,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,0x21a0adb6000000000000000000000000000000000000...,...,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaee...,NaN,NaN,NaN,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
4,15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed...,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,0x21a0adb6000000000000000000000000000000000000...,...,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e...,NaN,NaN,NaN,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83777,181253,0x5a36df89fa1f1ef6de7254e4b5fbfd2d6c2297ef4621...,164,188,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570447,70188284576,0xb438689f00000000000000000000000047ce0c6ed5b0...,...,1,2021-09-05 01:23:28 UTC,13162729,0xe14fb1085bc975af1ad751aa579d7de298fb5be86c3d...,1.027888e+11,2.210156e+09,2.0,70188284576,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x107753c195edde2fe69e573f667a697526178a47
83778,181255,0x43a081968e109f71a2f807f4a94ffc77429b322c3c1e...,169,168,0xe7a784acb145bd8993e83d41adf43b7c7195e995,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567923,39255417592,0xb438689f000000000000000000000000a160cdab2256...,...,1,2021-10-03 14:49:29 UTC,13346858,0xe5a4d9c829998f00873f6ac4603174d707ffb2ea79a3...,4.000000e+10,1.130179e+09,2.0,39255417592,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xe7a784acb145bd8993e83d41adf43b7c7195e995
83779,181262,0xd0708b3e2135d0378a9d6fe3fdc58b3c4fbcbcbe926c...,246,10,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,567971,50193886615,0xb438689f00000000000000000000000047ce0c6ed5b0...,...,1,2021-10-17 23:36:24 UTC,13438417,0x36808310975d158e0f2b3a2f892abed25a955684f407...,7.518635e+10,1.519911e+09,2.0,50193886615,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xa85f4669b31ad9c75adbfd288c37b678ab0b0fa4
83780,181263,0x54b5ceca9fd3d2ff7893c32f901f3d1d5e2e4383607b...,127,180,0xd2ddee09e723d165233986e0ed0aa6da1cea97e0,0x722122df12d4e14e13ac3b6895a86e84145b6967,0,570459,81048817558,0xb438689f00000000000000000000000047

In [7]:
deposit_transactions_df = pd.read_csv("../data/complete_deposit_txs.csv")

In [8]:
lighter_deposit_transactions_df = deposit_transactions_df.drop(columns=["input"])

In [9]:
lighter_deposit_transactions_df.to_csv("../data/lighter_complete_deposit_txs.csv")